In [93]:
# Import TensorFlow and hub
import tensorflow_hub as hub

# some important packages
import numpy as np
import pandas as pd

# scikit-learn
import joblib
from sklearn.neighbors import NearestNeighbors


In [94]:
model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(model_url)
print('Model Loaded')

Model Loaded


In [95]:
def embed(texts):
    return model(texts)


In [96]:
def mapping(path):
    try :
        file = open("../"+path, "r")
        content = file.read()
        if content == "":
            return np.NaN
        return content.replace("\n"," ")
    except FileNotFoundError :
        return np.NaN 
         
df = pd.read_csv("../data_mod.csv")
df["Content"] = df['Poem'].apply(mapping)
df.dropna(subset=["Content"],inplace=True)
df.drop(columns=["Punctuation_Count","empty_lines_count"],inplace=True)
df.drop_duplicates(subset=["Content"],inplace=True)
df.reset_index(drop=True, inplace=True)

In [97]:

pd.to_pickle(df, "data_for_recommandations.pkl")

df = pd.read_pickle("data_for_recommandations.pkl")

In [98]:
df

,Title,Author,Poem,Tags,Category,Content
0,tires,JAN WAGNER,data\CitiesUrban\CitiesUrban_1.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",CitiesUrban,where the city starts to come undone at the ra...
1,Anna May Wong has Breakfast at Tiffany’s,SALLY WEN MAO,data\CitiesUrban\CitiesUrban_2.txt,"['LIVING', 'DEATH', 'TIME & BREVITY', 'ARTS & ...",CitiesUrban,In Santa Monica the sunrise has this way of em...
2,Pilsen,SALIMA RIVERA,data\CitiesUrban\CitiesUrban_3.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",CitiesUrban,Late night sounds and smells entice and we pro...
3,"Los Angeles, Fin de Siècle",MAURYA SIMON,data\CitiesUrban\CitiesUrban_4.txt,"['LIVING', 'TIME & BREVITY', 'ARTS & SCIENCES'...",CitiesUrban,Delirium in the downtown mall today Burt Reyno...
4,A Toothless Crackhead Was the Mascot,REGINALD DWAYNE BETTS,data\CitiesUrban\CitiesUrban_5.txt,"['LIVING', 'COMING OF AGE', 'DEATH', 'LIFE CHO...",CitiesUrban,An Outline for a Film A woman leans against a ...
...,...,...,...,...,...,...
2190,ABC for Refugees,MONICA SOK,data\Youth\Youth_195.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",Youth,who doesn t read English well know that cherub...
2191,Crushing,SHANE BOOK,data\Youth\Youth_196.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",Youth,We smuggle crates of Beaujolais through the sq...
2192,CXIV,SONNET L'ABBÉ,data\Youth\Youth_197.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",Youth,I m not sure whether it happened in Manitoba o...
2193,Everybody Believes They Are the Good Guy,CYNTHIA ARRIEU-KING,data\Youth\Youth_198.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",Youth,I was hanging with grandparents in a kindergar...


In [99]:
poems = list(df['Content'])
embeddings = embed(poems)
print('The embedding shape is:', embeddings.shape)

The embedding shape is: (2195, 512)


In [100]:
nn = NearestNeighbors(metric='cosine', n_neighbors=5)
nn.fit(embeddings)

NearestNeighbors(metric='cosine')

In [101]:
def recommend(text: str):
    text.replace("\n", " ")
    emb = embed([text])
    neighbors = nn.kneighbors(emb, return_distance=False)[0]
    return df.iloc[neighbors]


In [102]:
print('Recommended poems:')
recommend("to my mom")

Recommended poems:


,Title,Author,Poem,Tags,Category,Content
921,Maybe my most important identity is being a son,RAYMOND ANTROBUS,data\Love\Love_124.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",Love,my mother to open a tab on her laptop calling ...
1370,What did you learn here? (Old Man House),CEDAR SIGO,data\Nature\Nature_97.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",Nature,For Joy Harjo
812,Infant Sorrow,WILLIAM BLAKE,data\Infancy\Infancy_70.txt,"['LIVING', 'INFANCY', 'RELATIONSHIPS', 'FAMILY...",Infancy,My mother groand my father wept Into the dange...
1232,Chewing slowly,KABIR,data\Marriage\Marriage_191.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",Marriage,god my darling do me a favour and kill my moth...
464,Changeling,HIEU MINH NGUYEN,data\Failure\Failure_84.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",Failure,Standing in front of a mirror my mother tells ...


In [103]:
recommend("love")

,Title,Author,Poem,Tags,Category,Content
1014,How God Answers the Soul,MECHTHILD OF MAGDEBURG,data\Love\Love_2_47.txt,"['LOVE', 'RELIGION', 'GOD & THE DIVINE', 'THE ...",Love,It is my nature that makes me love you often I...
947,Song,LOUISE BOGAN,data\Love\Love_155.txt,"['LOVE', 'REALISTIC & COMPLICATED', 'RELATIONS...",Love,Love me because I am lost Love me that I am un...
580,Ode to Facemasks,XI CHUAN,data\HistoryPolitics\HistoryPolitics_2.txt,"['LIVING', 'LIFE CHOICES', 'ARTS & SCIENCES', ...",HistoryPolitics,February
941,On Love,KAHLIL GIBRAN,data\Love\Love_149.txt,"['RELATED VIDEO', 'RELATED VIDEO', 'LOVE', 'RO...",Love,Then said Almitra Speak to us of Love And he r...
1194,"Sonnet 142: Love is my sin, and thy dear virtu...",WILLIAM SHAKESPEARE,data\Marriage\Marriage_150.txt,"['LIVING', 'MARRIAGE & COMPANIONSHIP', 'LOVE',...",Marriage,Love is my sin and thy dear virtue hate Hate o...


In [104]:
recommend("so much depends upon a red wheel barrow glazed with rain water beside the white chickens")

,Title,Author,Poem,Tags,Category,Content
1349,Never Describe the Sky as Azure,PHILIP METRES,data\Nature\Nature_70.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",Nature,unless it s made of stone migrating from Persi...
124,L.A. River,JACK COOPER,data\CitiesUrban\CitiesUrban_133.txt,"['NATURE', 'ANIMALS', 'SOCIAL COMMENTARIES', '...",CitiesUrban,I like how the mallard ducklings goofy and wea...
743,The Pretty Papers Section,ED ROBERSON,data\HistoryPolitics\HistoryPolitics_197.txt,"['APPEARED IN POETRY MAGAZINE', 'APPEARED IN P...",HistoryPolitics,dithers and bumstead trying to vote who gets t...
39,Morning Catalog,JOSEPH MASSEY,data\CitiesUrban\CitiesUrban_42.txt,"['NATURE', 'SOCIAL COMMENTARIES', 'CITIES & UR...",CitiesUrban,That sound s the ocean s with traffic ground d...
1344,Daybreak,BERT MEYERS,data\Nature\Nature_64.txt,['NATURE'],Nature,Birds drip from the trees The moon s a little ...


In [105]:
joblib.dump(nn, '5nn_model.joblib')
nn = joblib.load('5nn_model.joblib')